In [115]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_logs = pd.read_csv('input/train_logs.csv')
train_scores = pd.read_csv('input/train_scores.csv')

In [119]:
train_scores2 = train_scores.copy()
train_scores2['essay'] = ''
for ts_index, ts_row in train_scores2.iterrows():
    if ts_index % 100 == 1:
        print(ts_index)
    df = train_logs[train_logs['id'] == ts_row['id']].reset_index()
    essay = ''
    for index, row in df.iterrows():
        pos = row['cursor_position']
        old_pos = df.iloc[index - 1]['cursor_position'] if index != 0 else 0
        txt = row['text_change']
        if row['activity'] == 'Input':
            if pos - 1 == len(essay):
                essay += txt[0]
            else:
                essay = essay[:pos - 1] + txt[0] + essay[pos - 1:]
        elif row['activity'] == 'Remove/Cut':
            essay = essay[:pos] + essay[pos:].replace(txt, '', 1)
        elif row['activity'] == 'Paste':
            if old_pos == len(essay):
                essay += txt
            else:
                essay = essay[:old_pos] + txt + essay[old_pos:]
        elif row['activity'] == 'Replace':
            old, new = txt.split(' => ')
            essay = essay[:old_pos] + essay[old_pos:].replace(old, new, 1)
        elif row['activity'][:4] == 'Move':
            m = re.match(r'.*\[(\d+).*\[(\d+)', row['activity'])
            old_index, new_index = int(m.group(1)), int(m.group(2))
            essay = essay[:old_index] + essay[old_index:].replace(txt, '', 1)
            essay = essay = essay[:new_index] + txt + essay[new_index:]
    train_scores2['essay'][ts_index] = essay
train_scores2

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401


,id,score,essay
0,001519c8,3.5,qqqqqqqqq qq qqqqq qq qqqq qqqq. qqqqqq qqq q...
1,0022f953,3.5,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,6.0,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,2.0,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,4.0,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."
...,...,...,...
2466,ffb8c745,3.5,"qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq..."
2467,ffbef7e5,4.0,"qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq..."
2468,ffccd6fd,1.5,qqqqqq qqqq q qqqqqqq qqqqqqqqq qq qqqqqq qqqq...
2469,ffec5b38,5.0,"qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq..."


In [120]:
train_scores2.to_csv('output/essays.csv')